Computational Linguistics &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; <br>Biao Yun

<center> 

# Word Embeddings Assignment
### 2023.03.20

</center>

#### 引入所需 packages

In [1]:
import pandas as pd
import re
from ckiptagger import data_utils
from ckiptagger import WS
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.tokenize.treebank import TreebankWordDetokenizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from tqdm import tqdm
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.preprocessing.text import Tokenizer
from numpy import asarray
from numpy import zeros
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras_preprocessing.sequence import pad_sequences
import io
from sklearn.feature_extraction.text import TfidfVectorizer

#### 載入 CKIP 斷詞系統

In [2]:

ws = WS("/Users/biaoyun/Documents/111 Spring Semester Gtaduated Institute/Data/data")



/Users/biaoyun/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/layers/legacy_rnn/rnn_cell_impl.py:909: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.LSTMCell` is deprecated and will be "
/Users/biaoyun/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1700: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


## 讀入中文文件

* 此檔案為華語男歌手 2013-2023 之間發布之歌曲歌詞，包含歌手、歌名、歌詞、網址與日期等欄位，由魔鏡歌詞網取得
* 由於 CKIP 斷詞系統不堪負荷，從原檔案八萬多首歌中隨機取出5000首歌作為此次模型訓練資料
* 模型分類目的為判斷是否為 positive or negative 的歌詞

In [3]:

df = pd.read_csv('/Users/biaoyun/Documents/111 Spring Semester Gtaduated Institute/Data/male_all_recent_10_years_song_lyrics_content_clean.csv', sep='\t')
df = df.iloc[29585:34585, :]


In [4]:
df.head()

,singer,name,lyric,song_url,date
29585,高山林,五月花開,"作詞：孫浩, 作曲：孫浩, 終於盼到了滿園春色遍地花開, 你說這個季節會回來, 滿天飛舞的花...",twy131563x1x4.htm,2014-05
29586,高山林,兄弟保重,"作詞：周兵, 作曲：石焱, 讓我這杯酒 為你送行, 太多的話 都在杯裡, 人生注定有短暫分離...",twy131563x1x3.htm,2014-05
29587,高山林,你不懂愛,"作詞：阿博, 作曲：阿博, 我問了很久, 是什麼原由, 讓我們走不到盡頭, 我等了很久, 你...",twy131563x1x5.htm,2014-05
29588,高山林,乘風破浪,"作詞：小白, 作曲：小白, 揚帆啟航 海岸天際, 仰望蒼穹 鳥瞰大地, 生命如歌 瑰麗如虹,...",twy131563x1x11.htm,2014-05
29589,高山林,愛你必須的,"作詞：周兵, 作曲：石焱, 回憶漸漸開始飄落, 牽過你的手還記得, 美夢簡單複製了快樂, 存...",twy131563x1x2.htm,2014-05


## Pre-processing

* 首先將歌詞欄位的「中文」提取出來（即刪掉英文或是其他語言之歌詞）
* 接著將歌詞用 CKIP 進行斷詞（這邊真的跑超～～久，我原本想用一萬首，結果 kernel 當掉兩次😨只好換成五千首）
* 再來引入 LIWC 繁體中文情緒詞典，對歌詞進行簡單的正負相情緒判別，產生 sentiment 欄位，即之後分類模型的 label
* 正負向情緒給定方式為計算每首歌所含有之正向詞與負向詞，相減後大於 0 為正向，小於 0 為負向，等於 0 為中性
* 為減少模型雜訊與負擔，將情緒為中性之歌詞刪除
* 正向之歌詞給定 label 為 0，負向則為 1

In [5]:
# 提取中文字之 function

def extract_chinese(text):
    pattern = re.compile("[\u4e00-\u9fa5]")
    
    return "".join(pattern.findall(text))

In [6]:
ly_list = list(df['lyric'])

In [7]:
new_ly = []

for i in range(len(ly_list)):
    new_ly.append(extract_chinese(ly_list[i]))

In [8]:
df['lyric_new'] = new_ly

#### 斷詞

In [9]:

token_list = []
for text in tqdm(new_ly):
    tokens = ws([text])
    token_list.append(tokens)
# for text in df["lyric"]:
#   tokens = ws([text])
#   token_list.append(tokens)
    

100%|██████████| 5000/5000 [38:12<00:00,  2.18it/s]  


In [10]:

flatten_tokens = []
for list_1 in token_list:
  for sublist in list_1:
    flatten_tokens.append(sublist)
    

#### 計算歌詞之情緒向度

In [11]:
# sentiment

LIWC = pd.read_csv("/Users/biaoyun/Documents/111 Spring Semester Gtaduated Institute/Data/LIWC.csv", sep=',')

pos = [31]
neg = [32]
for num_31 in pos:
    pos_df = LIWC[LIWC.isin([num_31]).any(1)].reset_index(drop=True)
for num_32 in neg:
    neg_df = LIWC[LIWC.isin([num_32]).any(1)].reset_index(drop=True)

In [12]:
negemo_list = []

for neg in neg_df['%']:
    negemo_list.append(neg)

In [13]:
posemo_list = []

for pos in pos_df['%']:
    posemo_list.append(pos)

In [14]:
positive = []
negative = []
neutral = []
for tokens in token_list:
  pos_counter = 0
  neg_counter = 0
  neu = 0
  for i in tokens[0]:
    if i in posemo_list:
      pos_counter = pos_counter+1
  for i in tokens[0]:
    if i in negemo_list:
      neg_counter = neg_counter+1
  if pos_counter == 0 and neg_counter == 0:
    neu = 1
  else:
    new = 0
  positive.append(pos_counter)
  negative.append(neg_counter)
  neutral.append(neu)

In [15]:
emo = []
for tokens in token_list:
  pos_count = 0
  neg_count = 0
  for i in tokens[0]:
    if i in posemo_list:
      pos_count = pos_count+1
    if i in negemo_list:
      neg_count = neg_count+1
  if pos_count == 0 and neg_count == 0:
    result = 2
  elif pos_count-neg_count > 0:
    result = 0
  elif pos_count-neg_count < 0:
    result = 1
  emo.append(result)

In [16]:
df["sentiment"] = emo

In [17]:
flatten_string = []

for ft in flatten_tokens:
    st = ' '.join(ft)
    flatten_string.append(st)

In [18]:
df['segmented_lyrics'] = flatten_string

#### 刪除情緒為中性之歌詞

In [19]:
df_clean = df[df.sentiment != 2]

In [20]:
# 儲存檔案，才不會每次都要重新斷詞

df_clean.to_csv('/Users/biaoyun/Documents/111 Spring Semester Gtaduated Institute/Week_5/df_clean_new.csv', sep = '\t', index = False)

## (1) Create a tf-idf matrix with real Chinese dataset. (30%)

* 這邊的步驟為引入已經斷詞完成的欄位 ['segmented_lyrics']
* 計算 tf-idf
* 將字詞提請出來
* 繪製成 tf-idf 表格


In [20]:
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(df_clean['segmented_lyrics'])
feature_names = vectorizer.get_feature_names_out()

In [21]:
print(vectors.shape)

(4425, 41111)


In [22]:
df_tfidf = pd.DataFrame(vectors.toarray(),columns=feature_names)
df_tfidf

,一一,一丁,一三三兩兩三三,一下,一下來,一下子,一不可,一世,一世富一世,一世界,...,龔德中,龔敬文,龔曉然,龔淑均,龔言脩,龔高飛,龜殼,龜毛,龜派,龜蛋
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4420,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4421,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4422,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4423,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## (2) Use the Chinese dataset to create your own embeddings and train a classification model with a neural network model. (30%)

* 首先，先製作自己的 embeddings
* 建構模型，並將 embeddings 放在第一層，進行模型訓練
* 模型輸出維度為 10
* 模型評估方式為 cross entropy 之 loss function 與 accuracy
* 使用 cross entropy 是為了看預測與實際分佈之間的「距離」相差多少，即「損失了多少」（損失越小越好）
* 使用 accuracy 而非 f-score 是因為想要純粹看模型「完全正確」，即 TP 與 TN 之判斷成果


#### embeddings 製作

In [26]:
docs = list(df_clean['segmented_lyrics'])

In [27]:
labels = list(df_clean['sentiment'])

In [28]:
docs_strings = ' '.join(docs)

In [29]:
vocabs = set(docs_strings.split(' '))

In [30]:
vocab_size = len(vocabs) + 1

In [31]:
from keras.preprocessing.text import one_hot

vocab_size = len(vocabs)
hashed_docs = [one_hot(d, vocab_size) for d in docs]
print(hashed_docs[0:3])

[[35670, 5993, 24738, 5993, 28460, 34985, 34640, 3342, 4714, 6283, 4825, 33649, 15538, 34286, 38977, 12125, 18264, 4199, 37715, 31819, 3342, 11624, 33383, 33880, 29539, 33148, 37227, 33880, 15967, 7291, 34286, 17292, 33210, 7268, 23042, 33880, 14382, 39966, 42171, 18264, 16333, 27724, 38230, 17292, 34618, 22491, 17820, 17545, 12343, 34286, 18005, 12125, 3342, 4714, 7285, 19298, 36605, 15394, 34286, 29057, 9893, 34286, 37715, 25883, 33975, 18787, 17513, 17806, 17388, 1544, 19924, 12125, 39251, 34640, 33880, 19665, 38006, 27322, 43454, 1829, 15538, 31459, 17292, 4713, 12343, 34286, 32185, 24889, 17806, 33880, 13604, 32051, 20913, 18264, 18108, 33880, 17050, 10963, 29627, 34286, 9662, 17806, 33880, 25032, 41917, 29627, 34286, 8086, 43454, 1829, 7268, 23042, 42171, 11624, 42171, 11624, 33880, 12343, 7166, 34286, 31819, 5896, 33210, 39002, 15967, 38222, 30207, 30704, 38064, 37715, 8006, 37722, 38222, 30207, 33210, 39002, 22836, 20313, 17292, 34543, 33495, 28460, 34985, 34640, 3342, 4714, 62

In [32]:
max_length = max([len(d.split()) for d in docs])
print(max_length)

948


In [33]:
# from keras.utils import pad_sequences

padded_docs = pad_sequences(hashed_docs, maxlen=max_length, padding='post')
print(padded_docs[0:5])

[[35670  5993 24738 ...     0     0     0]
 [35670 16777 24738 ...     0     0     0]
 [35670 39609 24738 ...     0     0     0]
 [35670 25827 24738 ...     0     0     0]
 [35670 16777 24738 ...     0     0     0]]


#### 模型訓練（包含模型搭建、加入前面訓練的 embeddings、列印 model summary 與 model evaluation）

In [34]:

# define the model
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=max_length)) 
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(padded_docs, np.array(labels), epochs=50, verbose=0)

# summarize the model
print(model.summary())

# model evaluation
loss, accuracy = model.evaluate(padded_docs, np.array(labels), verbose=0)
print('Accuracy: %f' % (accuracy*100))
print('Loss: %d' % loss)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 948, 10)           436580    
_________________________________________________________________
flatten (Flatten)            (None, 9480)              0         
_________________________________________________________________
dense (Dense)                (None, 1)                 9481      
Total params: 446,061
Trainable params: 446,061
Non-trainable params: 0
_________________________________________________________________
None
Accuracy: 99.977404
Loss: 0


## 第二題 Reflection

單純用 embeddings 訓練的 accuracy 竟然能達到 99.98！推測是因為資料本身詞彙量夠多，且分類之標籤（positive or negative sentiment）和歌詞本身息息相關，故能夠有如此好的效果。另外，其實我一開始用的是前幾週 yahoo 影評的資料，結果訓練出來結果非常差，accuracy 只有十幾，loss 是負數，所以我才猜測單獨使用 embeddings 的模型表現和文本本身的特性非常相關。因為影評普遍非常短、包含大量平常認為的 stop words 以及標點符號、表情符號等等，不利 word embeddings 的分類任務。

## (3) Train the classification model with training set and test set. (20%)

* 首先，因為歌詞資料爬取時是照著注音符號之順序，故在分 training set 與 test set 之前先進行 shuffle
* 分為 training set 與 test set
* 建構模型並放入前面自製的 embeddings 做分類任務
* 評估方式比照前一題之模型

#### shuffle data 並區分 training and test set

In [35]:

shuffled_feature, shuffled_label = shuffle(padded_docs, labels, random_state = 888)
feature_train, feature_test, label_train, label_test = train_test_split(shuffled_feature, shuffled_label, test_size = 0.2, random_state = 413)



#### 模型訓練（包含模型搭建、加入前面訓練的 embeddings、列印 model summary 與 model evaluation）

In [36]:
# define the model

model_new = Sequential()
model_new.add(Embedding(vocab_size, 10, input_length=max_length)) 
model_new.add(Flatten())
model_new.add(Dense(1, activation='sigmoid'))

# compile the model

model_new.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_new.fit(feature_train, np.array(label_train), epochs = 50, verbose = 0)


# summarize the model

print(model_new.summary())

# model evaluation

loss_new, accuracy_new = model_new.evaluate(feature_test, np.array(label_test), verbose=0)
print('Accuracy: %f' % (accuracy_new*100))
print('Loss: %d' % loss_new)


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 948, 10)           436580    
_________________________________________________________________
flatten_1 (Flatten)          (None, 9480)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 9481      
Total params: 446,061
Trainable params: 446,061
Non-trainable params: 0
_________________________________________________________________
None
Accuracy: 83.276838
Loss: 0


## 第三題 Reflection

訓練完後會發現雖然模型的 accuracy 還算高，但對比前一題沒有區分訓練與測試集的模型來說低了非常多。我猜是因為如果沒有區分的話訓練的資料也會等於要預測的資料，故模型很有可能會 overfitting（有點像是拿著答案找答案的感覺）。所以在做模型訓練時區分訓練集以及測試集是非常重要的，比較能看到模型的真正表現以及要改進的地方。

## (4) Find an English dataset and train a model with pre-trained GloVe embeddings. Don't update the weight during training. And split the dataset into training set and test set before training a model. (20%)

* 首先，讀入英文資料
* 此資料由 Kaggle 取得，為奈及利亞工作招聘之廣告，其中包含真實與欺騙的廣告內容
* 廣告為真實之 label 為 0，反之為 1，故此次模型分類目的為判斷廣告知真假
* 進行斷詞
* 接著下載 GloVe 預訓練好的 embeddings (6B, 100 dim)
* 製作 embeddings matrix 和權重
* 建構模型，於第一層 embeddings 放入 GloVe 資料，將 trainable 參數設置為 False，避免每次模型自行更新權重
* shuffle 資料並區分 training and test set
* 評估模型方式照前面

#### 讀入英文資料

In [37]:

df_eng = pd.read_csv('/Users/biaoyun/Documents/111 Spring Semester Gtaduated Institute/Data/job_posting.csv', sep=',')
df_eng.head()



,job_title,company_name,company_desc,job_desc,job_requirement,salary,location,employment_type,department,label
0,Accountant,Equity Model Limited,"Accounting, Auditing & Finance","Compiling, analyzing, and reporting financial ...",This position is open preferably to a male can...,"75,000 - 150,000",Abuja,Full Time,Law & Compliance,0
1,Content Writer,CLINTON FUND (CF),Management & Business Development,"Creating, improving and maintaining content to...","Bachelor's degree in Journalism, English, Com...","60,000 - 100,000",Lagos,Full Time,Content Writing,1
2,Accountant,Schleez Nigeria Limited,"Accounting, Auditing & Finance","Managing financial transactions, preparing fin...",Minimum of Bachelor's degree in Accounting or ...,Negotiable,"First Floor, Left Wing, No. 49, Olowu Street, ...",Full-time,Accounting,1
3,Sales Executive,Bons Industries Limited,Marketing & Communications,"Understanding of the sales process and dynamics.""",Minimum academic qualification of BSC/HND Degr...,"75,000 - 150,000",Enugu,Full Time,Manufacturing & Warehousing,0
4,Bulk/Partnership Marketing Officer,TAMAK LOGISTICS,Marketing & Communications,Establish relationships with major businesses ...,"Be smart & resourceful.,Great knowledge of how...","Less than 75,000",Lagos,Full Time,Shipping & Logistics,0


In [38]:
job_desc = list(df_eng['job_desc'])
job_desc[0:3]

['Compiling, analyzing, and reporting financial data.',
 'Creating, improving and maintaining content to achieve our business goals. Your duties will also include sharing content to raise brand awareness and monitoring web traffic and metrics to identify best practices. ',
 'Managing financial transactions, preparing financial reports, and providing financial advice to clients.']

In [39]:
labels_eng = list(df_eng['label'])

#### 英文斷詞、製作 one-hot encodings

In [40]:
t = Tokenizer()
t.fit_on_texts(job_desc)

vocab_size_eng = len(t.word_index) + 1
hashed_sents = t.texts_to_sequences(job_desc)
print(hashed_sents[0:5])

[[329, 208, 1, 41, 9, 116], [149, 330, 1, 35, 75, 2, 150, 13, 14, 98, 331, 332, 76, 333, 334, 335, 75, 2, 209, 336, 337, 1, 210, 338, 339, 1, 340, 2, 62, 341, 211], [36, 9, 63, 64, 9, 46, 1, 77, 9, 151, 2, 6], [342, 4, 3, 24, 58, 1, 343], [212, 17, 5, 344, 345, 346, 18, 3, 213]]


In [41]:
max_length_eng = max([len(s.split()) for s in job_desc])
print(max_length_eng)

115


In [42]:
# pad documents to a max length
padded_sents = pad_sequences(hashed_sents, maxlen=max_length_eng, padding='post')
print(padded_sents[0:5])

[[329 208   1  41   9 116   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [149 330   1  35  75   2 150  13  14  98 331 332  76 333 334 335  75   2
  209 336 337   1 210 338 339   1 340   2  62 341 211   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [ 36   9  63  64   9  46   1  77   9 151   2   6 

#### 引入下載好的 GloVe pre-trained embeddings

In [43]:

embeddings_index = dict()

with open('/Users/biaoyun/Documents/111 Spring Semester Gtaduated Institute/Data/glove/glove.6B.100d.txt', encoding='utf-8') as f:
  for line in f:
    values = line.split()
    word = values[0]
    coefs = asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs

print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


#### 製作 weight matrix

In [44]:
# create a weight matrix for words in training docs

embedding_matrix = zeros((vocab_size_eng, 100))
for word, i in t.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

#### shuffle data 並區分 training and test set

In [45]:

shuffled_feature_eng, shuffled_label_eng = shuffle(padded_sents, labels_eng, random_state = 888)
feature_train_eng, feature_test_eng, label_train_eng, label_test_eng = train_test_split(shuffled_feature_eng, shuffled_label_eng, test_size = 0.2, random_state = 413)



#### 模型訓練（包含模型搭建、加入前面訓練的 embeddings、列印 model summary 與 model evaluation）

In [46]:
model_eng = Sequential()
e = Embedding(vocab_size_eng, 100, weights=[embedding_matrix], input_length= max_length_eng, trainable=False)
model_eng.add(e)
model_eng.add(Flatten())
model_eng.add(Dense(1, activation='sigmoid'))

# compile the model
model_eng.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_eng.fit(feature_train_eng, np.array(label_train_eng), epochs = 50, verbose = 0)

# summarize the model
print(model_eng.summary())

# model evaluation

loss_eng, accuracy_eng = model_eng.evaluate(feature_test_eng, np.array(label_test_eng), verbose=0)
print('Accuracy: %f' % (accuracy_eng*100))
print('Loss: %d' % loss_eng)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 115, 100)          83800     
_________________________________________________________________
flatten_2 (Flatten)          (None, 11500)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 11501     
Total params: 95,301
Trainable params: 11,501
Non-trainable params: 83,800
_________________________________________________________________
None
Accuracy: 85.365856
Loss: 0


## 第四題 Reflection

英文的資料雖然沒有用自製的 word embeddings，但是顯然 GloVe pre-trained embeddings 表現良好，accuracy 達到 85.4，loss 為零，所以證明可以用 pre-trained embeddings 達到想要的預測目標。且這個的好處是不用每次都要自己訓練、擔心有 overfitting 的問題，可以應用於不同內容的文本都有著不錯的表現。

## BONUS-1
### play around with the embedding visualization

* 這邊要可視化的是中文、有分 training and test set 的模型
* 取出第一層 embeddings 的資料（權重）
* 列出所有 words
* 分別存成 Embedding Projector 可以讀取之檔案
* 可視化 (成果為附件之影片檔)

* 取出 embedding 層每個 checkpoint 的 權重

In [47]:
emb_layer = model_new.layers[0]
weights = emb_layer.get_weights()[0]
print(weights.shape)


(43658, 10)


* 列出 word list

In [53]:
words = list(vocabs)

#### 分別存成 tsv 檔，以便上傳至 Embedding Projector 來進行可視化

In [49]:
i = 0

weight_vec = io.open('weight_vec.tsv', 'w', encoding='utf-8')
word_arr = io.open('words.tsv', 'w', encoding='utf-8')
for word in words:
    word_vec = weights[i]
    word_arr.write(word + "\n")
    weight_vec.write('\t'.join([str(x) for x in word_vec]) + '\n')
    i=i+1

weight_vec.close()
word_arr.close()

## BONUS-2
### play around with the GPU server and train with a larger GloVe embeddings

* 這邊下載了最大的 GloVe embeddings (840 B, 300 dim)
* 比照前述方式進行模型訓練
* p.s. 原本要試試看 Word2Vec 之類的，結果 gensim pacakage 和 numpy 版本衝突，升級 numpy 之後換 numpy 和 tensorflow 衝突......只好放棄><

In [50]:

embeddings_index_lar = dict()

with open('/Users/biaoyun/Documents/111 Spring Semester Gtaduated Institute/Data/glove.840B.300d.txt', encoding='utf-8') as f:
  for line in f:
    
    values = line.strip().split(' ')
    word = values[0]
    coefs = asarray(values[1:], dtype='float32')
    embeddings_index_lar[word] = coefs

print('Loaded %s word vectors.' % len(embeddings_index_lar))

Loaded 2196007 word vectors.


In [51]:
# create a weight matrix for words in training docs

embedding_matrix_lar = zeros((vocab_size_eng, 300))
for word, i in t.word_index.items():
    embedding_vector_lar = embeddings_index_lar.get(word)
    if embedding_vector_lar is not None:
        embedding_matrix_lar[i] = embedding_vector_lar

In [52]:
model_eng_lar = Sequential()
e_lar = Embedding(vocab_size_eng, 300, weights=[embedding_matrix_lar], input_length= max_length_eng, trainable=False)
model_eng_lar.add(e_lar)
model_eng_lar.add(Flatten())
model_eng_lar.add(Dense(1, activation='sigmoid'))

# compile the model
model_eng_lar.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_eng_lar.fit(feature_train_eng, np.array(label_train_eng), epochs = 50, verbose = 0)

# summarize the model
print(model_eng_lar.summary())

# model evaluation

loss_eng_lar, accuracy_eng_lar = model_eng_lar.evaluate(feature_test_eng, np.array(label_test_eng), verbose=0)
print('Accuracy: %f' % (accuracy_eng_lar*100))
print('Loss: %d' % loss_eng_lar)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 115, 300)          251400    
_________________________________________________________________
flatten_3 (Flatten)          (None, 34500)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 34501     
Total params: 285,901
Trainable params: 34,501
Non-trainable params: 251,400
_________________________________________________________________
None
Accuracy: 80.487806
Loss: 0


## BONUS-2 Reflection

費盡千辛萬可下載最大的 GloVe embeddings（檔案非常大），結果表現沒有 6 B 100 dim 的 embeddings 好 ><。我猜是因為資料太多且維度也過多，導致模型在訓練時有很多雜訊影響其判斷，才會降低了一些效果。因此，我建議在本身 dataset 沒有很大的時候用普通大小的 word embeddings 就足夠了。另外，補充個小資訊，'glove.840B.300d.txt' 裡面有不正常的符號，所以在轉換成 32 float 的時候會出現報錯，解決方式就是在 readlines 的時候加入 strip( ) 把沒用的符號去除掉才能夠正常運行。